In [2]:
from pandas_plink import read_plink
import pandas as pd
import dask
import dask_ml.preprocessing
import numpy as np

In [3]:
(bim_test, fam_test, G_test) = read_plink('/Users/jenniferblanc/polygenic_adaptation_stratification/output/Simulate_Genotypes/4PopSplit/S2/C1/genos-test_common')
(bim_gwas, fam_gwas, G_gwas) = read_plink('/Users/jenniferblanc/polygenic_adaptation_stratification/output/Simulate_Genotypes/4PopSplit/S2/C1/genos-gwas_common')

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 17.96it/s]


In [4]:
# Mean Center Test
X = G_test.transpose()
scaler = dask_ml.preprocessing.StandardScaler(with_mean=True, with_std=False).fit(X)
X = scaler.transform(X)

# Mean Center GWAS
M = G_gwas.transpose()
scaler = dask_ml.preprocessing.StandardScaler(with_mean=True, with_std=False).fit(M)
M = scaler.transform(M)

In [7]:
# Calculate Test covariance matrix 
test_cov = dask.array.cov(X)

# SVD of covariance matrix 
u, s, v = dask.array.linalg.svd(test_cov)
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
vals = dask.array.diag(1/s[0:(n-1)])
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose()).compute()

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)
Tm.compute()

type(stvec)

numpy.ndarray

In [9]:
Tvec = np.loadtxt("/Users/jenniferblanc/polygenic_adaptation_stratification/output/Calculate_Tm/4PopSplit/S2/Tvec.txt")

In [13]:
Tvec.shape

(200,)